In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# API Key Generator

In [7]:
from dataset.vr_scraper import APIKeyGenerator

key_gen = APIKeyGenerator()
key_gen.get_new_key()

Regenerating new key...


'SNefQmMbk0tUmgpnWsSoLSQq17GdP59tXQYQPy85HwM='

# Virtual Regatta Scraper

In [11]:
from dataset.vr_scraper import VRScraper

PLAYER_ID = "5fb42fba06999515a3c11088"
scraper = VRScraper(key_gen, PLAYER_ID)

## 1 - Get list of races

In [4]:
race_list = scraper.get_race_list(min_id=400, max_id=550)
race_list

Regenerating new key...
Regenerating new key...
Regenerating new key...


[(440, 'Imoca'),
 (442, 'Class 40'),
 (466, 'Multi 50'),
 (467, 'Class 40'),
 (468, 'Super Maxi 100'),
 (469, 'Class 40'),
 (470, 'Tara'),
 (479, 'Class 40')]

There seem to be only a (very) few races available in the database...

In [8]:
RACE_ID = 440 # Vendée Globe 2020

## 2 - Get race info

In [10]:
vg_infos = scraper.get_race_infos(RACE_ID)
vg_infos.keys()

NameError: name 'scraper' is not defined

In [7]:
vg_infos["course"][:5]

[{'lat': 46.4713, 'lon': -1.83153},
 {'lat': 42.673, 'lon': -28.2027},
 {'lat': -0.474495, 'lon': -26.9887},
 {'lat': -39.9364, 'lon': -27.0269},
 {'lat': -42.0177, 'lon': 14.595}]

In [8]:
vg_infos["checkpoints"][:2]

[{'id': 1,
  'group': 1,
  'name': 'Biscay',
  'start': {'lat': 43.055303984131, 'lon': -9.2993141089844},
  'end': {'lat': 42.07638696875, 'lon': -70.71201553125},
  'engine': True,
  'display': 'none',
  'side': 'port'},
 {'id': 2,
  'group': 2,
  'name': '',
  'start': {'lat': 1.6267552220703, 'lon': -49.912141628906},
  'end': {'lat': 14.74079, 'lon': -17.529111354492},
  'engine': True,
  'display': 'none',
  'side': 'stbd'}]

In [43]:
#vg_infos["ice_limits"] # should be used for binary map

In [9]:
import pickle
import os

VG_INFOS_FILE = f"infos_{RACE_ID}.pickle"

with open(VG_INFOS_FILE, 'wb') as f:
    pickle.dump(vg_infos, f)

### 3 - Get list of players 

In [12]:
NUM_PLAYERS = 100_000 # First 10.000 players
vg_player_list = scraper.get_player_list(RACE_ID, NUM_PLAYERS)
len(vg_player_list)

Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...


101000

In [13]:
vg_player_list[:3] # This is indeed the podium !

[{'_id': '59c253eeb395b292ed622a41',
  'distance': 0,
  'time': 5955364000,
  'displayName': 'tigrou26120',
  'country': 'FR',
  'rank': 1},
 {'_id': '59df528ce0750e372d63a217',
  'distance': 0,
  'time': 5955459000,
  'displayName': 'Tournesol II',
  'country': 'DE',
  'rank': 2},
 {'_id': '5e748753080c8cbd450a816d',
  'distance': 0,
  'time': 5955514000,
  'displayName': '*Kerjouanno*',
  'country': 'FR',
  'rank': 3}]

### 4- Get player infos

First player

In [12]:
vg_first_boat = scraper.get_boat_infos(RACE_ID, vg_player_list[0]["_id"])
vg_first_boat

Regenerating new key...


{'bs': {'_id': {'user_id': '59c253eeb395b292ed622a41',
   'race_id': 440,
   'leg_num': 1},
  'boat': {'name': 'Imoca',
   'label': 'Imoca',
   'polar_id': 4,
   'assetBundle': 'vendeeglobe',
   'stats': {'length': 18.28,
    'width': 5.85,
    'airDraught': 29,
    'draught': 4.5,
    'weight': 8000,
    'upwindSails': 300,
    'downwindSails': 600},
   'type': 'imoca',
   'skin_id': 'imoca_real_corum',
   'locked': True,
   'display': {'color1': '#ffffff', 'color2': '#ffffff'}},
  'branding': {'name': 'Virtual Regatta', 'id': 'VR'},
  'priceLevel': 1,
  'credits': 2400,
  'legStartDate': 1604836920000,
  'options': ['skin',
   'hull',
   'winch',
   'foil',
   'light',
   'heavy',
   'reach',
   'radio'],
  'type': 'sponsor',
  'earlyBirdLimit': None,
  'isMultiLeg': False,
  'usedCodes': ['corum6', 'hucaxheerb'],
  'personal': {'country': 'FR',
   'deptId': 3020781,
   'townId': 3027705,
   'yachtClubId': None},
  'startDate': 1603475711840,
  'displayName': 'tigrou26120',
  'prefer

In [13]:
from datetime import datetime

ts1 = vg_first_boat["track"]["track"][0]['ts'] # First record
ts2 = vg_first_boat["track"]["track"][1]['ts'] # Second record
print(datetime.fromtimestamp(ts1/1000)) # year, month, day, hour, minute
print(datetime.fromtimestamp(ts2/1000))

2020-11-08 13:02:00
2020-11-08 15:40:00


First `NUM_PLAYERS` players

In [15]:
import pickle
import os

vg_player_ids = [p["_id"] for p in vg_player_list]
VG_BOATS_FILE = f"boats_{RACE_ID}_{NUM_PLAYERS}.pickle"

if os.path.exists(VG_BOATS_FILE):
    print("File exists!")
    with open(VG_BOATS_FILE, 'rb') as f:
        vg_boats = pickle.load(f)
else:  
    # This is VERY long
    vg_boats = scraper.get_boat_infos_bulk(RACE_ID, vg_player_ids)
    with open(VG_BOATS_FILE, 'wb') as f:
        pickle.dump(vg_boats, f)

  0%|          | 0/101000 [00:00<?, ?it/s]

Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Regenerating new key...
Failed to get API Key!  => Retry (1/5)
Regenerating new key...
Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=87.0.4280.141)
  => Retry (2/5)
Regenerating new key...
Message: invalid session id
  => Retry (3/5)
Regenerating new key...
Message: invalid session id
  => Retry (4/5)
Regenerating new key...
Message: invalid session id
  => Retry (5/5)
Regenerating new key...
Message: invalid session id
  => Retry (1/5)
Regenerating new key...
Message: invalid session id
  => Retry (2/5)
Regenerating new key.

KeyError: 'res'